In [11]:
from prettyprinter import pprint
import polars as pl
import asyncio
import httpx

In [12]:
series_ids = [
    "STEO.RGDPQ_NONOECD.M",
    "STEO.RGDPQ_OECD.M",
    "STEO.PAPR_NONOPEC.M",
    "STEO.PAPR_OPEC.M",
    "STEO.PATC_OECD.M",
    "STEO.PATC_NON_OECD.M",
    "STEO.COPRPUS.M",
    "STEO.CORIPUS.M",
    "STEO.FOREX_WORLD.M",
    "STEO.PASC_OECD_T3.M",
    "STEO.COPS_OPEC.M",
    "STEO.COPC_OPEC.M",
    "STEO.T3_STCHANGE_OOECD.M",
    "STEO.T3_STCHANGE_NOECD.M"
]

X_Params = {
    "frequency": "monthly",
    "data": [
        "value"
    ],
    "facets": {},
    "start": "2000-01",
    "end": "2024-01",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
    "offset": 0,

}

API_KEY = "caGyhsgCC3brWOYqKDFMtPc16VPABFzHr3PIfHf7"

In [13]:
async def fetch_eia_data(series_id: str, headers: dict = {}):
    url = f"https://api.eia.gov/v2/seriesid/{series_id}?api_key={API_KEY}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        response.raise_for_status()  
        return response.json()


async def get_all_data (list_of_series_id: list, headers: dict = {}):
    tasks = [fetch_eia_data(series_id, headers) for series_id in list_of_series_id]
    return await asyncio.gather(*tasks)


responses = await get_all_data(list_of_series_id=series_ids,headers=X_Params)
DATA = list(map(lambda x: pl.DataFrame(x['response']['data']), filter(lambda x: len(x['response']['data']), responses)))

In [9]:
metadata = {
    "series_id": list(map(lambda x: x['seriesId'][0], DATA)),
    "description": list(map(lambda x: x['seriesDescription'][0], DATA)),
    "unit": list(map(lambda x: x['unit'][0], DATA)),
    "n_observations": list(map(lambda x: x.shape[0], DATA)),
    }


In [10]:
print(pl.DataFrame(metadata).to_pandas().to_markdown(index=False))

| series_id         | description                                                             | unit                           |   n_observations |
|:------------------|:------------------------------------------------------------------------|:-------------------------------|-----------------:|
| PAPR_NONOPEC      | Total non-OPEC Liquids Petroleum Production                             | million barrels per day        |              396 |
| PAPR_OPEC         | Total OPEC Petroleum Supply                                             | million barrels per day        |              396 |
| PATC_OECD         | Liquid Fuels Consumption, Total OECD                                    | million barrels per day        |              432 |
| PATC_NON_OECD     | Liquid Fuels Consumption, Total non-OECD                                | million barrels per day        |              432 |
| COPRPUS           | U.S. Crude Oil Production                                               | million barrels 

In [16]:
import pickle
with open("DATA.pkl", "wb") as f:
    pickle.dump(list(map(lambda x: x.to_pandas(), DATA)), f)

In [17]:
with open("DATA.pkl", "rb") as f:
    DATA_TEST = pickle.load(f)